## Instalation des depandances.

In [1]:
# ! pip install transformers

In [121]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from torch import cuda

### Bert Tokenizer

In [122]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [123]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [124]:
MAX_LEN = 50
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05

## Les Donnees

In [125]:
train_data_path = "./Version 1/data/data_questions/questions-train.txt"
test_data_path =  "./Version 1/data/data_questions/questions-test.txt"

### Les classes de nos questions

In [126]:
classes = {
     'ABBREVIATION': 0,
     'DEFINITION': 1,
     'DESCRIPTION': 2,
     'ENTITY': 3,
     'LOCATION': 4,
     'ORGANIZATION': 5,
     'PERSON': 6,
     'QUANTITY': 7,
     'TEMPORAL': 8
}

In [127]:
def get_data(datapath):
    X = []
    Y = []
    with open(datapath, 'r') as file:
        lines = file.readlines()
        for line in lines:
            splited_line = line.split(" ")
            Y.append(classes[splited_line[0]])
            X.append(" ".join(splited_line[1:]))
    return X , Y

In [128]:
X , Y = get_data(train_data_path)
sentences = [tokenizer.tokenize(sentence) for sentence in X]

#### Trouver la taille de la question la plus long

In [129]:
# sentence_max_length = 0 

# for sentence in sentences:
#     sentence_max_len = max(sentence_max_len, len(sentence))

In [130]:
# def mask_inputs_for_bert(sentences ,sentence_max_length):
#     # Tokenize all of the sentences and map the tokens to thier word IDs.
#     input_ids = []
#     attention_masks = []
#     i = 0 
#     for sentence in sentences:
#         # “encode_plus* will:
#         # (1) Tokenize the sentence.
#         # (2) Prepend the “[CLS]" token to the start.
#         # (3) Append the “[SEP]* token to the end.
#         # (4) Map tokens to their IDs.
#         # (5) Pad or truncate the sentence to ‘max_length*while i < len(sentence):
#         # (6) Create attention masks for [PAD] tokens.

#         if(i<3):
#             print("Question", sentence)

#             encoded_dict = tokenizer.encode_plus(
#                             sentence, # Sentence to encode.
#                             add_special_tokens = True, # Add ‘[CLS]' and '[SEP]’
#                             max_length = sentence_max_len, # Pad & truncate all sentences.
#                             pad_to_max_length = True,
#                             return_attention_mask = True, # Construct attn. masks.
#                             )
#         if(i<3):

#             print("dict",encoded_dict['input_ids'])
#             # Add the encoded sentence to the list.

#             input_ids.append(encoded_dict['input_ids'])

#             # and its attention mask (simply differentiates padding from non-padding).
#             attention_masks.append(encoded_dict['attention_mask'])

#             i = i+1

#     #         convert to tensor and return

#             input_ids = torch.tensor(input_ids)
#             attention_masks = torch.tensor(attention_masks)
#             return input_ids,attention_masks


In [131]:
# X_train, train_mask = mask_inputs_for_bert(X , sentence_max_length)# In[ ]:def test_get_all_by_host_with_pool(self):
# y_train = torch.tensor(Y)

In [132]:
# bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

In [133]:
# loss = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params =  bert_model.parameters(), lr=1e-05)

In [134]:
# bert_model.compile(loss=loss, optimizer=optimizer)

In [135]:
# def loss_fn(outputs, targets):
#     return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [136]:
# optimizer = torch.optim.Adam(params =  bert_model.parameters(), lr=1e-05)

In [137]:
# def train(epoch):
#     bert_model.train()
#     for _,data in enumerate(training_loader, 0):
#         ids = data['ids'].to(device, dtype = torch.long)
#         mask = data['mask'].to(device, dtype = torch.long)
#         token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#         targets = data['targets'].to(device, dtype = torch.float)

#         outputs = model(ids, mask, token_type_ids)

#         optimizer.zero_grad()
#         loss = loss_fn(outputs, targets)
#         if _%5000==0:
#             print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

In [160]:
class CustomDataset(Dataset):

    def __init__(self, X, y, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.sentences = X
        self.targets = y
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        question = self.sentences[index]
        inputs = self.tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [161]:
training_set = CustomDataset(X, Y, tokenizer, MAX_LEN)

In [162]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

In [163]:
training_loader = DataLoader(training_set, **train_params)

In [164]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
#         self.l2 = torch.nn.Dropout(0.3)
#         self.l3 = torch.nn.Linear(768, 9)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
#         output_2 = self.l2(output_1)
#         output = self.l3(output_2)
        return output_1

In [165]:
model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [180]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [181]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [182]:
for epoch in range(EPOCHS):
    train(epoch)

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not str